In [3]:
from pydantic import BaseModel, Field
from typing import Optional

class Person(BaseModel):
    """Extracted information about a person"""
    name: Optional[str] = Field(..., description="The name of the person")
    age: Optional[int] = Field(..., description="The age of the person")
    job: Optional[str] = Field(..., description="The job the person does")

In [11]:
from openai import OpenAI

class ExtractionLLM:
    def __init__(self, datamodel: BaseModel):
        self.client = OpenAI()
        self.datamodel = datamodel

    def extract(self, text: str) -> BaseModel:
        resposnse = self.client.beta.chat.completions.parse(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "Extract the following information from the text. If the information is not present, please set the field to None."},
                {"role": "user", "content": text}
            ],
            response_format=Person
        )
        return resposnse.choices[0].message.parsed


In [12]:
llm = ExtractionLLM(Person)

In [13]:
llm.extract("The person is a 30 year old software engineer named John Doe.")

Person(name='John Doe', age=30, job='software engineer')

In [10]:
llm.extract("Tha cat sat on the mat.")

Person(name=None, age=None, job=None)